In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models, callbacks


DATA_DIR = '/content/drive/MyDrive/dataset'
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 50


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.10,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.3
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Model definition
model = models.Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer='l2'),
    layers.Dropout(0.3),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stop]
)


model.save('emoji_cnn_model.h5')


def predict_emoji(image_path, model, class_indices):
    img = load_img(image_path, target_size=IMAGE_SIZE, color_mode='grayscale')
    arr = img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    preds = model.predict(arr)
    idx = np.argmax(preds, axis=1)[0]
    inv_map = {v: k for k, v in class_indices.items()}
    return inv_map[idx], preds[0][idx]

if __name__ == '__main__':
    from tensorflow.keras.models import load_model
    clf = load_model('emoji_cnn_model.h5')
    class_map = train_gen.class_indices



Found 378 images belonging to 18 classes.
Found 162 images belonging to 18 classes.
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 430ms/step - accuracy: 0.0450 - loss: 4.9065 - val_accuracy: 0.0741 - val_loss: 3.7492
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 216ms/step - accuracy: 0.0603 - loss: 3.5420 - val_accuracy: 0.0494 - val_loss: 3.0886
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 230ms/step - accuracy: 0.0738 - loss: 3.0296 - val_accuracy: 0.0988 - val_loss: 2.8826
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 213ms/step - accuracy: 0.0974 - loss: 2.8436 - val_accuracy: 0.1543 - val_loss: 2.6354
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.1708 - loss: 2.6186 - val_accuracy: 0.2654 - val_loss: 2.4592
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 259ms/step - accuracy: 0.2233 - loss: 2.4812 - val_accuracy: 0.2469 - val_loss: 2.3447
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step - accuracy: 0.2923 - loss: 2.2901 - val_accuracy: 0.3642 - val_loss: 2.1127
Epoch 8/50
12/12 ━━━